In [1]:
import pandas as pd
from src.dimension_modeler import Dimension_Modeler
from src.util import update_start_date, quick_merge

### FACT TRANSACTIONS
- transaction_key (int) PK (Surrogate Key)
- transaction_id (int) (Natural Key) DONE
- employee_id (int) FK to dim_employees DONE
- product_key (int) FK to dim_products DONE
- date_id (int) FK to dim_dates DONE
- promotion_id (int) FK to dim_promotions DONE
- payment_method_id (int) FK to dim_payment_methods DONE
- quantity (int) DONE
- discount_amount (decimal 10,2) DONE
- subtotal_amount (decimal 10,2) DONE
- tax_amount (decimal 10,2) DONE
- total_sale_amount (decimal 10,2) DONE

In [2]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [ ]:
# dim_transactions = Dimension_Modeler(transactions_df)
# # doing this because I have to restart the interpreter and don't want to run the code to add natural key again
# dim_transactions.make_csv('fact_transactions')

fact_transactions.csv created in datasets/dimensions/


#### Transaction id (natrual key)

In [3]:
grouped_df = transactions_df.groupby('Date', group_keys=False)
id_counter = 100

In [4]:
df_list = []
for name, _ in grouped_df:
    current_df = grouped_df.get_group(name).copy()
    current_df['transaction_id'] = id_counter
    id_counter += 1
    df_list.append(current_df)
    
transactions_df = pd.concat(df_list)

#### Employee ID


In [6]:
employee_df = pd.read_csv('datasets/fact_dimensions_models/dim_employees.csv')

In [7]:
transactions_df = quick_merge(transactions_df, employee_df, 'left', 'Employee Name', 'employee_name')

In [8]:
transactions_df.drop(columns=['Employee Name', 'job_title', 'employee_name'], inplace=True)

In [9]:
transactions_df.head()

,Product Name,Category,Date,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Promo Name,Quantity,Price per Unit,transaction_id,employee_id
0,Starbucks Mocha,Drinks,2015-01-01 00:56:32,5.00,0.31,5.31,Card,0.0,BOGO,4,1.25,100,4
1,Coffee,Drinks,2015-01-01 00:56:32,2.50,0.16,2.66,Card,0.0,BOGO,2,1.25,100,4
2,Starbucks Caramel,Drinks,2015-01-01 00:56:32,3.75,0.23,3.98,Card,0.0,BOGO,3,1.25,100,4
3,Redbull,Drinks,2015-01-01 00:56:32,3.75,0.23,3.98,Card,0.0,BOGO,3,1.25,100,4
4,Strawberry,Donut,2015-01-01 01:35:44,1.00,0.06,1.06,Card,0.0,NaN,2,0.50,101,4


In [22]:
transactions_df.groupby('transaction_id')['Date'].value_counts()

transaction_id  Date               
100             2015-01-01 00:56:32    4
101             2015-01-01 01:35:44    1
102             2015-01-01 01:41:45    2
103             2015-01-01 01:50:21    1
104             2015-01-01 02:04:55    4
                                      ..
251959          2024-12-31 22:46:18    5
251960          2024-12-31 22:53:09    3
251961          2024-12-31 23:03:42    5
251962          2024-12-31 23:40:26    5
251963          2024-12-31 23:53:27    4
Name: count, Length: 251864, dtype: int64

In [23]:
transactions_df['transaction_id'].max()

251963

#### Product Key

In [10]:
products_df = pd.read_csv('datasets/fact_dimensions_models/dim_products.csv')

In [11]:
products_df.tail(10)


,product_key,product_id,product_name,product_category,product_price,product_cost,start_date,end_date,is_current
44,30,122,Sprite,Drinks,2.00,1.15,2019-12-31 23:49:08,2262-04-11 23:47:16,True
45,38,122,Sprite,Drinks,1.25,1.00,2015-01-01 00:56:32,2019-12-31 23:49:08,False
46,32,123,Starbucks Caramel,Drinks,2.00,1.15,2019-12-31 23:49:08,2262-04-11 23:47:16,True
47,40,123,Starbucks Caramel,Drinks,1.25,1.00,2015-01-01 00:56:32,2019-12-31 23:49:08,False
48,35,124,Starbucks Mocha,Drinks,2.00,1.15,2019-12-31 23:49:08,2262-04-11 23:47:16,True
49,43,124,Starbucks Mocha,Drinks,1.25,1.00,2015-01-01 00:56:32,2019-12-31 23:49:08,False
50,13,125,Strawberry,Donut,1.00,0.10,2019-12-31 23:49:08,2262-04-11 23:47:16,True
51,27,125,Strawberry,Donut,0.50,0.10,2015-01-01 01:35:44,2019-12-31 23:49:08,False
52,31,126,Tropicana,Drinks,2.00,1.15,2019-12-31 23:49:08,2262-04-11 23:47:16,True
53,39,126,Tropicana,Drinks,1.25,1.00,2015-01-01 00:56:32,2019-12-31 23:49:08,False


In [12]:
transactions_df = quick_merge(transactions_df, products_df, 'inner', ['Product Name', 'Price per Unit'], ['product_name', 'product_price'])

In [13]:
# spot checking to make sure it worked. 
transactions_df.loc[transactions_df['Product Name'] == 'Chocolate']['product_key'].value_counts()

product_key
5     17626
19    10656
Name: count, dtype: int64

In [14]:
transactions_df.columns

Index(['Product Name', 'Category', 'Date', 'Subtotal', 'Tax Amount',
       'Total Amount', 'Payment Method', 'Discount Amount', 'Promo Name',
       'Quantity', 'Price per Unit', 'transaction_id', 'employee_id',
       'product_key', 'product_id', 'product_name', 'product_category',
       'product_price', 'product_cost', 'start_date', 'end_date',
       'is_current'],
      dtype='object')

In [15]:
transactions_df.drop(columns=['product_id', 'product_name', 'product_category', 'start_date', 'end_date', 'Product Name', 'Category', 'Price per Unit', 'product_cost', 'is_current'], inplace=True)

In [16]:
transactions_df.head()

,Date,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Promo Name,Quantity,transaction_id,employee_id,product_key,product_price
0,2015-01-01 00:56:32,5.00,0.31,5.31,Card,0.00,BOGO,4,100,4,43,1.25
1,2015-01-01 07:42:43,1.88,0.12,1.99,Card,1.88,WINTER SPECIAL,3,110,2,43,1.25
2,2015-01-02 18:00:33,2.50,0.16,2.66,Card,0.00,NaN,2,171,1,43,1.25
3,2015-01-03 06:38:16,1.88,0.12,1.99,Cash,1.88,WINTER SPECIAL,3,196,5,43,1.25
4,2015-01-03 08:01:45,1.25,0.08,1.33,Cash,0.00,BOGO,1,199,3,43,1.25


#### DATE ID

In [17]:
transactions_df.head()


,Date,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Promo Name,Quantity,transaction_id,employee_id,product_key,product_price
0,2015-01-01 00:56:32,5.00,0.31,5.31,Card,0.00,BOGO,4,100,4,43,1.25
1,2015-01-01 07:42:43,1.88,0.12,1.99,Card,1.88,WINTER SPECIAL,3,110,2,43,1.25
2,2015-01-02 18:00:33,2.50,0.16,2.66,Card,0.00,NaN,2,171,1,43,1.25
3,2015-01-03 06:38:16,1.88,0.12,1.99,Cash,1.88,WINTER SPECIAL,3,196,5,43,1.25
4,2015-01-03 08:01:45,1.25,0.08,1.33,Cash,0.00,BOGO,1,199,3,43,1.25


In [24]:
transactions_df['Date'] = pd.to_datetime(transactions_df['Date'])
transactions_df['date_raw'] = transactions_df['Date'].dt.strftime('%y-%m-%d')
transactions_df['date_raw'] = pd.to_datetime(transactions_df['date_raw'], format='%y-%m-%d')
transactions_df['date_id'] = transactions_df['date_raw'].dt.year.astype(str) + transactions_df['date_raw'].dt.month.astype(str).str.zfill(2) + transactions_df['date_raw'].dt.day.astype(str).str.zfill(2)
transactions_df['date_id'] = transactions_df['date_id'].astype(int)

In [25]:
transactions_df.head()

,Date,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Promo Name,Quantity,transaction_id,employee_id,product_key,product_price,date_raw,date_id
0,2015-01-01 00:56:32,5.00,0.31,5.31,Card,0.00,BOGO,4,100,4,43,1.25,2015-01-01,20150101
1,2015-01-01 07:42:43,1.88,0.12,1.99,Card,1.88,WINTER SPECIAL,3,110,2,43,1.25,2015-01-01,20150101
2,2015-01-02 18:00:33,2.50,0.16,2.66,Card,0.00,NaN,2,171,1,43,1.25,2015-01-02,20150102
3,2015-01-03 06:38:16,1.88,0.12,1.99,Cash,1.88,WINTER SPECIAL,3,196,5,43,1.25,2015-01-03,20150103
4,2015-01-03 08:01:45,1.25,0.08,1.33,Cash,0.00,BOGO,1,199,3,43,1.25,2015-01-03,20150103


In [26]:
transactions_df.drop(columns=['Date', 'date_raw'], inplace=True)

In [27]:
transactions_df.head()


,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Promo Name,Quantity,transaction_id,employee_id,product_key,product_price,date_id
0,5.00,0.31,5.31,Card,0.00,BOGO,4,100,4,43,1.25,20150101
1,1.88,0.12,1.99,Card,1.88,WINTER SPECIAL,3,110,2,43,1.25,20150101
2,2.50,0.16,2.66,Card,0.00,NaN,2,171,1,43,1.25,20150102
3,1.88,0.12,1.99,Cash,1.88,WINTER SPECIAL,3,196,5,43,1.25,20150103
4,1.25,0.08,1.33,Cash,0.00,BOGO,1,199,3,43,1.25,20150103


In [28]:
transactions_df.shape

(756135, 12)

#### Promotion ID


In [29]:
transactions_df['Promo Name'].value_counts()

Promo Name
BOGO              83971
WINTER SPECIAL    19988
Back to School     1440
Name: count, dtype: int64

In [30]:
transactions_df['Promo Name'] = transactions_df['Promo Name'].str.upper()

In [31]:
promotion_df = pd.read_csv('datasets/fact_dimensions_models/dim_promotions.csv')

In [32]:
promotion_df

,promotion_id,promotion_name,discount_percentage,description
0,1,BOGO,0.50,"Happens sporadically throughout the year, all ..."
1,2,WINTER SPECIAL,0.50,"During Dec, Jan, Feb, all products 50% off."
2,3,BACK TO SCHOOL,0.25,"During first week of September, 25% off all pr..."
3,4,NO PROMO,0.00,No promotion


In [33]:
transactions_df.isna().sum()

Subtotal                0
Tax Amount              0
Total Amount            0
Payment Method          0
Discount Amount         0
Promo Name         650736
Quantity                0
transaction_id          0
employee_id             0
product_key             0
product_price           0
date_id                 0
dtype: int64

In [34]:
transactions_df['Promo Name'] = transactions_df['Promo Name'].fillna('NO PROMO')

In [35]:
transactions_df.isna().sum()

Subtotal           0
Tax Amount         0
Total Amount       0
Payment Method     0
Discount Amount    0
Promo Name         0
Quantity           0
transaction_id     0
employee_id        0
product_key        0
product_price      0
date_id            0
dtype: int64

In [36]:
transactions_df = quick_merge(transactions_df, promotion_df, 'inner', 'Promo Name', 'promotion_name')

In [37]:
transactions_df.head()

,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Promo Name,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id,promotion_name,discount_percentage,description
0,5.00,0.31,5.31,Card,0.0,BOGO,4,100,4,43,1.25,20150101,1,BOGO,0.5,"Happens sporadically throughout the year, all ..."
1,1.25,0.08,1.33,Cash,0.0,BOGO,1,199,3,43,1.25,20150103,1,BOGO,0.5,"Happens sporadically throughout the year, all ..."
2,3.75,0.23,3.98,Card,0.0,BOGO,3,272,3,43,1.25,20150105,1,BOGO,0.5,"Happens sporadically throughout the year, all ..."
3,5.00,0.31,5.31,Cash,0.0,BOGO,4,671,4,43,1.25,20150115,1,BOGO,0.5,"Happens sporadically throughout the year, all ..."
4,5.00,0.31,5.31,Cash,0.0,BOGO,4,717,1,43,1.25,20150116,1,BOGO,0.5,"Happens sporadically throughout the year, all ..."


In [38]:
transactions_df.columns

Index(['Subtotal', 'Tax Amount', 'Total Amount', 'Payment Method',
       'Discount Amount', 'Promo Name', 'Quantity', 'transaction_id',
       'employee_id', 'product_key', 'product_price', 'date_id',
       'promotion_id', 'promotion_name', 'discount_percentage', 'description'],
      dtype='object')

In [39]:
transactions_df.drop(columns=['Promo Name', 'promotion_name', 'discount_percentage', 'description'], inplace=True)

In [40]:
transactions_df['promotion_id'].value_counts()

promotion_id
4    650736
1     83971
2     19988
3      1440
Name: count, dtype: int64

In [41]:
transactions_df.shape

(756135, 12)

In [42]:
transactions_df.sample(4)

,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id
23267,5.0,0.31,5.31,Card,0.0,4,17801,2,38,1.25,20160226,1
377728,4.0,0.25,4.25,Card,0.0,4,240185,8,6,1.00,20240922,4
569800,8.0,0.50,8.50,Card,0.0,4,207676,3,29,2.00,20231219,4
545966,6.0,0.38,6.38,Cash,0.0,3,115774,4,30,2.00,20210419,4


#### Payment Method ID

In [43]:
transactions_df.head()

,Subtotal,Tax Amount,Total Amount,Payment Method,Discount Amount,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id
0,5.00,0.31,5.31,Card,0.0,4,100,4,43,1.25,20150101,1
1,1.25,0.08,1.33,Cash,0.0,1,199,3,43,1.25,20150103,1
2,3.75,0.23,3.98,Card,0.0,3,272,3,43,1.25,20150105,1
3,5.00,0.31,5.31,Cash,0.0,4,671,4,43,1.25,20150115,1
4,5.00,0.31,5.31,Cash,0.0,4,717,1,43,1.25,20150116,1


In [44]:
transactions_df['Payment Method'].value_counts()

Payment Method
Card    536815
Cash    219320
Name: count, dtype: int64

In [45]:
transactions_df['Payment Method'].isnull().sum()

0

In [46]:
transactions_df.shape

(756135, 12)

In [47]:
payment_method_df = pd.read_csv('datasets/fact_dimensions_models/dim_payment_methods.csv')

In [48]:
payment_method_df

,payment_method_id,payment_method
0,1,Cash
1,2,Card


In [49]:
transactions_df = quick_merge(transactions_df, payment_method_df, 'inner', 'Payment Method', 'payment_method')

In [50]:
transactions_df.groupby('payment_method_id')['Payment Method'].value_counts()

payment_method_id  Payment Method
1                  Cash              219320
2                  Card              536815
Name: count, dtype: int64

In [51]:
transactions_df.columns

Index(['Subtotal', 'Tax Amount', 'Total Amount', 'Payment Method',
       'Discount Amount', 'Quantity', 'transaction_id', 'employee_id',
       'product_key', 'product_price', 'date_id', 'promotion_id',
       'payment_method_id', 'payment_method'],
      dtype='object')

In [52]:
transactions_df.drop(columns=['payment_method', 'Payment Method'], inplace=True)

In [53]:
transactions_df.head()

,Subtotal,Tax Amount,Total Amount,Discount Amount,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id,payment_method_id
0,5.00,0.31,5.31,0.0,4,100,4,43,1.25,20150101,1,2
1,3.75,0.23,3.98,0.0,3,272,3,43,1.25,20150105,1,2
2,3.75,0.23,3.98,0.0,3,804,1,43,1.25,20150118,1,2
3,2.50,0.16,2.66,0.0,2,852,3,43,1.25,20150119,1,2
4,3.75,0.23,3.98,0.0,3,964,2,43,1.25,20150122,1,2


#### Transaction key/Surrogate key

In [54]:
transactions_fact_model = Dimension_Modeler(transactions_df)

In [55]:
transactions_fact_model.make_id_col('transaction_key')

,transaction_key,Subtotal,Tax Amount,Total Amount,Discount Amount,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id,payment_method_id
0,1,5.00,0.31,5.31,0.00,4,100,4,43,1.25,20150101,1,2
1,2,3.75,0.23,3.98,0.00,3,272,3,43,1.25,20150105,1,2
2,3,3.75,0.23,3.98,0.00,3,804,1,43,1.25,20150118,1,2
3,4,2.50,0.16,2.66,0.00,2,852,3,43,1.25,20150119,1,2
4,5,3.75,0.23,3.98,0.00,3,964,2,43,1.25,20150122,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756130,756131,0.75,0.05,0.80,0.25,1,159826,6,9,1.00,20220904,3,1
756131,756132,3.00,0.19,3.19,1.00,4,159872,1,9,1.00,20220904,3,1
756132,756133,3.00,0.19,3.19,1.00,4,159929,7,9,1.00,20220905,3,1
756133,756134,2.25,0.14,2.39,0.75,3,238375,6,9,1.00,20240906,3,1


In [57]:
transactions_fact_model.df

,transaction_key,Subtotal,Tax Amount,Total Amount,Discount Amount,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id,payment_method_id
0,1,5.00,0.31,5.31,0.00,4,100,4,43,1.25,20150101,1,2
1,2,3.75,0.23,3.98,0.00,3,272,3,43,1.25,20150105,1,2
2,3,3.75,0.23,3.98,0.00,3,804,1,43,1.25,20150118,1,2
3,4,2.50,0.16,2.66,0.00,2,852,3,43,1.25,20150119,1,2
4,5,3.75,0.23,3.98,0.00,3,964,2,43,1.25,20150122,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756130,756131,0.75,0.05,0.80,0.25,1,159826,6,9,1.00,20220904,3,1
756131,756132,3.00,0.19,3.19,1.00,4,159872,1,9,1.00,20220904,3,1
756132,756133,3.00,0.19,3.19,1.00,4,159929,7,9,1.00,20220905,3,1
756133,756134,2.25,0.14,2.39,0.75,3,238375,6,9,1.00,20240906,3,1


#### Fixing columns etc to match the data model

In [2]:
transactions_final_df = pd.read_csv('datasets/fact_dimensions_models/fact_transactions.csv')

In [3]:
transactions_final_df.sample(10)

,transaction_key,Subtotal,Tax Amount,Total Amount,Discount Amount,Quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id,payment_method_id
231200,231201,1.0,0.06,1.06,0.0,2,89434,4,25,0.5,20191002,4,2
109992,109993,6.0,0.38,6.38,0.0,4,22943,3,51,1.5,20160615,4,2
110682,110683,3.0,0.19,3.19,0.0,2,34424,3,51,1.5,20170216,4,2
717487,717488,12.0,0.75,12.75,0.0,4,227949,2,45,3.0,20240610,4,1
590645,590646,4.5,0.28,4.78,0.0,3,26322,2,54,1.5,20160827,4,1
396191,396192,6.0,0.38,6.38,0.0,3,195593,1,29,2.0,20230822,4,2
602015,602016,1.5,0.09,1.59,0.0,3,79422,5,21,0.5,20190426,4,1
460075,460076,4.0,0.25,4.25,0.0,4,136830,3,10,1.0,20220109,4,2
367504,367505,4.0,0.25,4.25,0.0,4,105602,5,2,1.0,20201119,4,2
329189,329190,3.0,0.19,3.19,0.0,3,199178,7,13,1.0,20230926,4,2


In [5]:
transactions_final_df.duplicated().sum()

0

In [6]:
transactions_final_df.isnull().sum()

transaction_key      0
Subtotal             0
Tax Amount           0
Total Amount         0
Discount Amount      0
Quantity             0
transaction_id       0
employee_id          0
product_key          0
product_price        0
date_id              0
promotion_id         0
payment_method_id    0
dtype: int64

In [7]:
transactions_final_df.columns = transactions_final_df.columns.str.lower().str.replace(' ', '_')

In [8]:
transactions_final_df.head()

,transaction_key,subtotal,tax_amount,total_amount,discount_amount,quantity,transaction_id,employee_id,product_key,product_price,date_id,promotion_id,payment_method_id
0,1,5.00,0.31,5.31,0.0,4,100,4,43,1.25,20150101,1,2
1,2,3.75,0.23,3.98,0.0,3,272,3,43,1.25,20150105,1,2
2,3,3.75,0.23,3.98,0.0,3,804,1,43,1.25,20150118,1,2
3,4,2.50,0.16,2.66,0.0,2,852,3,43,1.25,20150119,1,2
4,5,3.75,0.23,3.98,0.0,3,964,2,43,1.25,20150122,1,2


In [10]:
transactions_final_df.rename(columns={'subtotal': 'subtotal_amount', 'total_amount': 'total_sale_amount'}, inplace=True)

In [11]:
transactions_final_df = transactions_final_df[['transaction_key', 'transaction_id', 'employee_id', 'product_key', 'date_id', 'promotion_id', 'payment_method_id', 'quantity', 'discount_amount', 'subtotal_amount', 'tax_amount', 'total_sale_amount']]

In [12]:
transactions_final_df.head()

,transaction_key,transaction_id,employee_id,product_key,date_id,promotion_id,payment_method_id,quantity,discount_amount,subtotal_amount,tax_amount,total_sale_amount
0,1,100,4,43,20150101,1,2,4,0.0,5.00,0.31,5.31
1,2,272,3,43,20150105,1,2,3,0.0,3.75,0.23,3.98
2,3,804,1,43,20150118,1,2,3,0.0,3.75,0.23,3.98
3,4,852,3,43,20150119,1,2,2,0.0,2.50,0.16,2.66
4,5,964,2,43,20150122,1,2,3,0.0,3.75,0.23,3.98


In [13]:
transactions_final_df.dtypes

transaction_key        int64
transaction_id         int64
employee_id            int64
product_key            int64
date_id                int64
promotion_id           int64
payment_method_id      int64
quantity               int64
discount_amount      float64
subtotal_amount      float64
tax_amount           float64
total_sale_amount    float64
dtype: object

In [14]:
transactions_fact_model = Dimension_Modeler(transactions_final_df)

In [15]:
transactions_fact_model.make_csv('fact_transactions')

fact_transactions.csv created in datasets/fact_dimensions_models/ folder.


### DIM DATES
- date_id (int) DONE
- full_date (date) DONE
- day (int) DONE
- month (int) DONE
- year (int) DONE
- quarter (int) DONE

#### Getting date ID 

In [ ]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [ ]:
transactions_df.head()

In [ ]:
transactions_df['Date'] = pd.to_datetime(transactions_df['Date'])

In [ ]:
transactions_df[['Date']].max()

In [ ]:
transactions_df[['Date']].min()

In [ ]:
transactions_df['date_raw'] = transactions_df['Date'].dt.strftime('%y-%m-%d')

In [ ]:
transactions_df[['date_raw']].min()

In [ ]:
transactions_df[['date_raw']].max()

In [ ]:
transactions_df['date_raw'] = pd.to_datetime(transactions_df['date_raw'], format='%y-%m-%d')

In [ ]:
transactions_df[['date_raw']].max()

In [ ]:
transactions_df[['date_raw']].min()

In [ ]:
transactions_df['date_raw'].dt.day

In [ ]:
transactions_df['date_id'] = transactions_df['date_raw'].dt.year.astype(str) + transactions_df['date_raw'].dt.month.astype(str).str.zfill(2) + transactions_df['date_raw'].dt.day.astype(str).str.zfill(2)

In [ ]:
transactions_df['date_id'] = transactions_df['date_id'].astype(int)

In [ ]:
transactions_df.dtypes

In [ ]:
transactions_df['date_raw'].nunique()

In [ ]:
transactions_df['date_id'].nunique()

In [ ]:
date_df = transactions_df.drop_duplicates('date_id', keep='first').copy()

In [ ]:
date_df['date_id'].count()

In [ ]:
transactions_df.groupby('date_id')['Date'].count().sort_values()

In [ ]:
transactions_df.loc[transactions_df['date_id'] == 20200125]

#### Making the date dataframe & extracting the rest of the information we need


In [ ]:
date_df = date_df[['date_raw', 'date_id']]

In [ ]:
date_df.duplicated().sum()

In [ ]:
date_df.dtypes

In [ ]:
date_df.rename(columns={'date_raw': 'full_date'}, inplace=True)

In [ ]:
date_df['day'] = date_df['full_date'].dt.day

In [ ]:
date_df.loc[:,'month'] = date_df['full_date'].dt.month

In [ ]:
date_df.loc[:,'year'] = date_df['full_date'].dt.year

In [ ]:
date_df.loc[:,'quarter'] = date_df['full_date'].dt.quarter

In [ ]:
date_df = date_df[['date_id', 'full_date', 'year', 'month', 'day', 'quarter']]
date_df

In [ ]:
date_dim_model = Dimension_Modeler(date_df)

In [ ]:
date_dim_model.make_csv('dim_dates')

### DIM PROMOTIONS
- promotion_id (int)
- promotion_name (str) DONE
- discount_percentage (decimal 5,2) DONE
- description (str) DONE

In [ ]:
promo_df = pd.read_csv('datasets/donut_shop_promos.csv')

In [ ]:
promo_df.head()

#### Conversion of discount percentage to float

In [ ]:
promo_df['Discount Perc'] = promo_df['Discount Perc'].apply(lambda x: x.split('%')[0])

In [ ]:
promo_df['Discount Perc'] = promo_df['Discount Perc'].astype(float)

In [ ]:
promo_df.dtypes

In [ ]:
promo_df['Discount Perc'] = promo_df['Discount Perc']/100

In [ ]:
promo_df

#### Adding id column, rearranging and fixing column names and saving to csv

In [ ]:
promo_dim_model = Dimension_Modeler(promo_df)

In [ ]:
promo_dim_model.make_id_col('promotion_id')

In [ ]:
promo_dim_model.df

In [ ]:
promo_dim_model.df.columns = promo_dim_model.df.columns.str.lower().str.replace(' ', '_')

In [ ]:
promo_dim_model.df.rename(columns={'promo_name': 'promotion_name', 'discount_perc': 'discount_percentage'}, inplace=True)

In [ ]:
promo_dim_model.df.dtypes

In [ ]:
new_row = pd.DataFrame([{'promotion_id': 4, 'promotion_name': 'NO PROMO', 'discount_percentage': 0, 'description': 'No promotion'}])
new_row

In [ ]:
promo_dim_model.df = pd.concat([promo_dim_model.df, new_row], ignore_index=True)

In [ ]:
promo_dim_model.df

In [ ]:
promo_dim_model.make_csv('dim_promotions')

### DIM PAYMENT METHODS
- payment_method_id (int) DONE
- payment_method (str) DONE

In [ ]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [ ]:
payment_method_df = transactions_df[['Payment Method']]
payment_method_df

In [ ]:
payment_method_df =payment_method_df.drop_duplicates(keep='first')

In [ ]:
payment_method_df.drop(columns='level_0', inplace=True)

In [ ]:
payment_method_df.columns = payment_method_df.columns.str.lower().str.replace(' ', '_')

In [ ]:
payment_method_df

In [ ]:
payment_method_dim_model = Dimension_Modeler(payment_method_df)

In [ ]:
payment_method_dim_model.make_id_col('payment_method_id')
payment_method_dim_model.df

In [ ]:
payment_method_dim_model.df.dtypes

In [ ]:
payment_method_dim_model.make_csv('dim_payment_methods')

### DIM PRODUCTS
- product_id (int) (Natural Key) DONE
- product_key (int) (Surrogate Key) DONE
- product_name (str) DONE
- product_category (str) DONE
- product_price (decimal 10,2) DONE
- product_cost (decimal 10,2) DONE 
- is_current(boolean) DONE
- start_date (date) DONE 
- end_date (date) DONE

In [ ]:
products_df = pd.read_csv('datasets/donut_shop_product_info.csv')

In [ ]:
products_df

#### Getting historical prices and dates associated

In [ ]:
transactions_df = pd.read_csv('datasets/donut_shop_transactions.csv')

In [ ]:
price_history = []


In [ ]:
for x in transactions_df.groupby('Category')['Price per Unit'].value_counts().index:
    price_history.append({
            'category': x[0],
            'price': x[1],
            'start_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].min(),
            'end_date': transactions_df[transactions_df['Price per Unit'] == x[1]]['Date'].max(),
            }
                         )

In [ ]:
price_history

In [ ]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].max()

In [ ]:
transactions_df[transactions_df['Price per Unit'] == 1.00]['Date'].min()

In [ ]:
dim_prod_df = pd.DataFrame(price_history)
    

#### Getting is_current field

In [ ]:
dim_prod_df

In [ ]:
dim_prod_df['max_start_date'] = dim_prod_df.groupby('category')['start_date'].transform('max')
dim_prod_df

In [ ]:
dim_prod_df['is_current'] = dim_prod_df['max_start_date'] == dim_prod_df['start_date']
dim_prod_df

In [ ]:
dim_prod_df.drop(columns='max_start_date', inplace=True)


#### Updating end_date

In [ ]:
dim_prod_df

In [ ]:
max_date = pd.Timestamp.max.strftime('%Y-%m-%d %H:%M:%S')
max_date

In [ ]:
dim_prod_df.loc[dim_prod_df['is_current'], 'end_date'] = max_date

#### Updating Start_date

In [ ]:
dim_prod_df

In [ ]:
dim_prod_df = dim_prod_df.groupby('category', group_keys=False).apply(update_start_date)

In [ ]:
dim_prod_df

#### Adding Historical prices


In [ ]:
dim_prod_df

In [ ]:
products_df

In [ ]:
dim_prod_df = pd.merge(dim_prod_df, products_df, how='left', left_on='category', right_on='Product Category')

In [ ]:
dim_prod_df = dim_prod_df[['category', 'price', 'start_date', 'end_date', 'is_current', 'Product Cost', 'Historical Cost']]

In [ ]:
dim_prod_df

In [ ]:
dim_prod_df.loc[~dim_prod_df['is_current'], 'Product Cost'] = dim_prod_df['Historical Cost']
dim_prod_df.drop(columns='Historical Cost', inplace=True)

In [ ]:
dim_prod_df

In [ ]:
products_df

#### Adding product name

In [ ]:
dim_prod_df

In [ ]:
products_list_df = transactions_df[['Product Name', 'Category']].drop_duplicates(keep='first').reset_index().drop(columns='index')

In [ ]:
products_list_df

In [ ]:
dim_prod_df = pd.merge(dim_prod_df, products_list_df, how='left', left_on='category', right_on='Category')


In [ ]:
dim_prod_df.drop(columns='Category', inplace=True)
dim_prod_df

#### Adding surrogate key / PK

In [ ]:
prod_dim_model = Dimension_Modeler(dim_prod_df)

In [ ]:
prod_dim_model.make_id_col('product_key')

#### Product ID creation (Natural key/business key)

In [ ]:
dim_prod_df = prod_dim_model.df

In [ ]:
dim_prod_df.head()

In [ ]:
grouped_df = dim_prod_df.groupby('Product Name', group_keys=False)

modified_groups = []

In [ ]:
id_counter = 100

for group_name, group_data in grouped_df:
    current_df = grouped_df.get_group(group_name).copy()
    current_df['product_id'] = id_counter
    id_counter += 1
    modified_groups.append(current_df)

combined_df = pd.concat(modified_groups)

In [ ]:
dim_prod_df = combined_df

In [ ]:
dim_prod_df.sort_values(by='product_id')


#### Fixing any dtypes and reordering columns & saving dim_products csv file.

In [ ]:
dim_prod_df.head()

In [ ]:
dim_prod_df.dtypes

In [ ]:
dim_prod_df['start_date'] = pd.to_datetime(dim_prod_df['start_date'])
dim_prod_df['end_date'] = pd.to_datetime(dim_prod_df['end_date'])

In [ ]:
dim_prod_df.head()

In [ ]:
dim_prod_df.columns = dim_prod_df.columns.str.lower().str.replace(' ', '_')
dim_prod_df.head()

In [ ]:
dim_prod_df.rename(columns={'category': 'product_category', 'price': 'product_price'}, inplace=True)

In [ ]:
dim_prod_df.head()

In [ ]:
prod_dim_model.df = dim_prod_df[['product_key', 'product_id', 'product_name', 'product_category', 'product_price', 'product_cost', 'start_date', 'end_date', 'is_current']]

In [ ]:
prod_dim_model.df.head()

In [ ]:
prod_dim_model.make_csv('dim_products')

### DIM EMPLOYEES
- employee_id (int) DONE 
- employee_name (str) DONE 
- job_title (str) DONE 

In [ ]:
employee_df = pd.read_csv('datasets/donut_shop_employees.csv')


In [ ]:
dim_model = Dimension_Modeler(employee_df)

In [ ]:
employee_df.columns = ['employee_name', 'job_title']

In [ ]:
employee_df.dtypes

In [ ]:
employee_df = dim_model.make_id_col('employee_id')

In [ ]:
employee_df.dtypes

In [ ]:
employee_df

In [ ]:
dim_model.make_csv('dim_employees')